In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv("Customers.csv")

In [5]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [6]:
print("Aperçu des données:")
display(df.head())

print("\nInformations sur le dataframe:")
print(f"Nombre de lignes: {df.shape[0]}")
print(f"Nombre de colonnes: {df.shape[1]}")
print("\nTypes de données:")
print(df.dtypes)

Aperçu des données:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes



Informations sur le dataframe:
Nombre de lignes: 7043
Nombre de colonnes: 21

Types de données:
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


## Reperage des valeurs nulles

In [7]:
# Traitement spécifique des valeurs vides dans TotalCharges

# 1. Identifier les lignes problématiques
empty_total_charges = df[df['TotalCharges'].astype(str).str.strip() == '']
print(f"Nombre de lignes avec TotalCharges vide: {len(empty_total_charges)}")

# Afficher ces lignes pour examiner
print("\nAperçu des lignes avec TotalCharges vide:")
display(empty_total_charges)

# 2. Vérifier la valeur de 'tenure' pour ces lignes
if 'tenure' in empty_total_charges.columns:
    print("\nDistribution de 'tenure' pour ces lignes:")
    print(empty_total_charges['tenure'].value_counts())

# 3. Examiner la relation entre tenure et ces lignes vides
if len(empty_total_charges) > 0 and 'tenure' in empty_total_charges.columns:
    # On s'attend à ce que les lignes avec TotalCharges vide soient des nouveaux clients
    print("\nVérification de l'hypothèse: TotalCharges vide => nouveaux clients")
    print(f"Minimum tenure: {empty_total_charges['tenure'].min()}")
    print(f"Maximum tenure: {empty_total_charges['tenure'].max()}")
    print(f"Médiane tenure: {empty_total_charges['tenure'].median()}")
    
    # Vérifier si tous les nouveaux clients (tenure=0 ou tenure=1) ont TotalCharges vide
    new_customers = df[(df['tenure'] == 0) | (df['tenure'] == 1)]
    new_with_empty = new_customers[new_customers['TotalCharges'].astype(str).str.strip() == '']
    
    print(f"\nNombre total de nouveaux clients (tenure 0-1): {len(new_customers)}")
    print(f"Nombre de nouveaux clients avec TotalCharges vide: {len(new_with_empty)}")
    print(f"Pourcentage: {len(new_with_empty) / len(new_customers) * 100:.2f}%")

# 4. Remplacer les valeurs vides
df_clean = df.copy()

# Convertir les TotalCharges vides en NaN
df_clean.loc[df_clean['TotalCharges'].astype(str).str.strip() == '', 'TotalCharges'] = np.nan

# Convertir la colonne en type numérique
df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')

# Pour les clients avec tenure=0, utilisez MonthlyCharges (ils viennent de s'inscrire)
zero_tenure_mask = df_clean['tenure'] == 0
df_clean.loc[zero_tenure_mask & df_clean['TotalCharges'].isna(), 'TotalCharges'] = \
    df_clean.loc[zero_tenure_mask & df_clean['TotalCharges'].isna(), 'MonthlyCharges']

# Pour les clients avec tenure>0, calculer TotalCharges = MonthlyCharges * tenure
# Ce n'est pas parfait, mais c'est une bonne approximation
higher_tenure_mask = (df_clean['tenure'] > 0) & df_clean['TotalCharges'].isna()
df_clean.loc[higher_tenure_mask, 'TotalCharges'] = \
    df_clean.loc[higher_tenure_mask, 'MonthlyCharges'] * df_clean.loc[higher_tenure_mask, 'tenure']

# Vérifier qu'il ne reste plus de valeurs manquantes
print(f"\nNombre de valeurs NaN restantes dans TotalCharges: {df_clean['TotalCharges'].isna().sum()}")

# Afficher les statistiques après correction
print("\nStatistiques de TotalCharges après correction:")
print(df_clean['TotalCharges'].describe())

# Vérifier la cohérence entre MonthlyCharges et TotalCharges
df_clean['CalculatedTotal'] = df_clean['MonthlyCharges'] * df_clean['tenure']
df_clean['TotalDifference'] = df_clean['TotalCharges'] - df_clean['CalculatedTotal']
df_clean['TotalDifferencePercent'] = (df_clean['TotalDifference'] / df_clean['TotalCharges']) * 100

print("\nStatistiques des différences entre TotalCharges et MonthlyCharges*tenure:")
print(df_clean[['TotalDifference', 'TotalDifferencePercent']].describe())

Nombre de lignes avec TotalCharges vide: 11

Aperçu des lignes avec TotalCharges vide:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No



Distribution de 'tenure' pour ces lignes:
tenure
0    11
Name: count, dtype: int64

Vérification de l'hypothèse: TotalCharges vide => nouveaux clients
Minimum tenure: 0
Maximum tenure: 0
Médiane tenure: 0.0

Nombre total de nouveaux clients (tenure 0-1): 624
Nombre de nouveaux clients avec TotalCharges vide: 11
Pourcentage: 1.76%

Nombre de valeurs NaN restantes dans TotalCharges: 0

Statistiques de TotalCharges après correction:
count    7043.000000
mean     2279.798992
std      2266.730170
min        18.800000
25%       398.550000
50%      1394.550000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64

Statistiques des différences entre TotalCharges et MonthlyCharges*tenure:
       TotalDifference  TotalDifferencePercent
count      7043.000000             7043.000000
mean          0.217642               -0.074585
std          67.228532                6.475492
min        -370.850000              -45.062837
25%         -28.600000               -2.080122
50%  

## Remplacage des valeurs nulles

In [8]:
# Correction définitive des valeurs vides dans TotalCharges

# 1. Identification des lignes problématiques
empty_total_charges = df[df['TotalCharges'].astype(str).str.strip() == '']
print(f"Avant correction: {len(empty_total_charges)} lignes avec TotalCharges vide")

# 2. Copie du dataframe pour les modifications
df_clean = df.copy()

# 3. Remplacement des valeurs vides par NaN puis conversion en numérique
df_clean.loc[df_clean['TotalCharges'].astype(str).str.strip() == '', 'TotalCharges'] = np.nan
df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')

# 4. Application des règles métier pour remplacer les NaN
# Pour les clients avec tenure=0, TotalCharges = MonthlyCharges
tenure_zero_mask = df_clean['tenure'] == 0
df_clean.loc[tenure_zero_mask & df_clean['TotalCharges'].isna(), 'TotalCharges'] = \
    df_clean.loc[tenure_zero_mask & df_clean['TotalCharges'].isna(), 'MonthlyCharges']

# 5. Vérification après correction
# Convertir temporairement en chaîne pour la vérification
still_empty = df_clean[df_clean['TotalCharges'].isna()]
print(f"Après correction: {len(still_empty)} lignes avec TotalCharges NaN")

# 6. Double vérification en recherchant des chaînes vides (pour être absolument sûr)
empty_check = df_clean[df_clean['TotalCharges'].astype(str).str.strip() == '']
print(f"Double vérification: {len(empty_check)} lignes avec TotalCharges contenant des chaînes vides")

# Si tout s'est bien passé, df_clean est maintenant prêt pour les étapes suivantes

# 7. Vérification de la distribution de TotalCharges
print("\nStatistiques de TotalCharges après correction:")
print(df_clean['TotalCharges'].describe())

# 8. Réassigner le dataframe corrigé à df pour les étapes suivantes
# Important: cette étape assure que les modifications sont conservées
df = df_clean.copy()

# 9. Vérification finale
final_check = df[df['TotalCharges'].astype(str).str.strip() == '']
print(f"\nVérification finale: {len(final_check)} lignes avec TotalCharges contenant des chaînes vides")

Avant correction: 11 lignes avec TotalCharges vide
Après correction: 0 lignes avec TotalCharges NaN
Double vérification: 0 lignes avec TotalCharges contenant des chaînes vides

Statistiques de TotalCharges après correction:
count    7043.000000
mean     2279.798992
std      2266.730170
min        18.800000
25%       398.550000
50%      1394.550000
75%      3786.600000
max      8684.800000
Name: TotalCharges, dtype: float64

Vérification finale: 0 lignes avec TotalCharges contenant des chaînes vides


In [9]:
duplicates_all = df.duplicated().sum()
print(f"Nombre de doublons (lignes identiques): {duplicates_all}")

# Vérifier les doublons basés uniquement sur customerID
if 'customerID' in df.columns:
    duplicates_id = df.duplicated(subset=['customerID']).sum()
    print(f"Nombre de customerID dupliqués: {duplicates_id}")
    
    # Afficher les customerID dupliqués si présents
    if duplicates_id > 0:
        duplicate_ids = df[df.duplicated(subset=['customerID'], keep=False)]
        print("\nLignes avec customerID dupliqués:")
        display(duplicate_ids)

# Recherche avancée de doublons partiels
# Ignorer customerID et vérifier si des clients ont des caractéristiques identiques
if 'customerID' in df.columns:
    columns_without_id = [col for col in df.columns if col != 'customerID']
    partial_duplicates = df.duplicated(subset=columns_without_id).sum()
    print(f"\nNombre de lignes avec caractéristiques identiques (hors customerID): {partial_duplicates}")
    
    # Afficher les doublons partiels si présents
    if partial_duplicates > 0:
        partial_dups = df[df.duplicated(subset=columns_without_id, keep=False)]
        print("\nExemples de lignes avec caractéristiques identiques:")
        display(partial_dups.head(10))

Nombre de doublons (lignes identiques): 0
Nombre de customerID dupliqués: 0

Nombre de lignes avec caractéristiques identiques (hors customerID): 22

Exemples de lignes avec caractéristiques identiques:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
22,1066-JKSGK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes
100,6380-ARCEH,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.20,20.20,No
542,2866-IKBTM,Female,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.55,19.55,No
646,0887-HJGAR,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,45.70,45.70,Yes
662,3030-ZKIWL,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.05,20.05,No
690,8262-COGGB,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,20.45,20.45,No
964,9117-SHLZX,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,45.70,45.70,Yes
976,5875-YPQFJ,Male,0,No,No,1,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,69.90,69.90,Yes
1243,5552-ZNFSJ,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,45.30,45.30,Yes
1338,1934-SJVJK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes


In [10]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

## Convertion

In [11]:
df_clean = df.copy()

# Conversion de TotalCharges en float
if 'TotalCharges' in df_clean.columns:
    print("Conversion de TotalCharges en type numérique...")
    # Remplacer les espaces par NaN puis convertir en float
    df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')
    
    # Vérifier si des valeurs ont été converties en NaN
    na_count = df_clean['TotalCharges'].isna().sum()
    print(f"Nombre de valeurs NaN après conversion: {na_count}")
    
    # Si des valeurs NaN ont été créées, les inspecter
    if na_count > 0:
        print("Lignes avec TotalCharges non numériques:")
        display(df[df_clean['TotalCharges'].isna()])

# Conversion de SeniorCitizen en catégorie (Yes/No)
if 'SeniorCitizen' in df_clean.columns:
    print("\nConversion de SeniorCitizen en Yes/No...")
    df_clean['SeniorCitizen'] = df_clean['SeniorCitizen'].map({1: 'Yes', 0: 'No'})
    
    # Vérifier la conversion
    print(df_clean['SeniorCitizen'].value_counts())

# Afficher les types de données après conversion
print("\nTypes de données après conversion:")
print(df_clean.dtypes)

Conversion de TotalCharges en type numérique...
Nombre de valeurs NaN après conversion: 0

Conversion de SeniorCitizen en Yes/No...
SeniorCitizen
No     5901
Yes    1142
Name: count, dtype: int64

Types de données après conversion:
customerID           object
gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object


In [12]:

# 1. Encodage des variables binaires (Yes/No -> 1/0)
binary_columns = [
    'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn'
]

binary_available = [col for col in binary_columns if col in df_clean.columns]

if binary_available:
    print("Encodage des variables binaires (Yes/No -> 1/0):")
    for col in binary_available:
        if df_clean[col].dtype == 'object' and set(df_clean[col].unique()).issubset({'Yes', 'No', np.nan}):
            df_clean[col] = df_clean[col].map({'Yes': 1, 'No': 0})
            print(f"  {col}: encodage terminé")
        else:
            print(f"  {col}: déjà encodé ou format différent")

# 2. Identifier toutes les variables catégorielles restantes
categorical_columns = df_clean.select_dtypes(include=['object']).columns.tolist()
print(f"\nVariables catégorielles à encoder avec one-hot: {categorical_columns}")

# 3. Appliquer le one-hot encoding
df_encoded = pd.get_dummies(
    df_clean, 
    columns=[col for col in categorical_columns if col != 'customerID'],
    drop_first=True
)

# 4. Afficher en détail toutes les colonnes avant et après encodage
print("\n" + "="*50)
print("COMPARAISON DÉTAILLÉE DES COLONNES AVANT/APRÈS ENCODAGE")
print("="*50)

# Colonnes originales
print("\nA. COLONNES ORIGINALES:")
original_cols = df_clean.columns.tolist()
print(f"Nombre total: {len(original_cols)}")

# Afficher les colonnes par type de données
for dtype in ['object', 'int64', 'float64', 'category']:
    type_cols = df_clean.select_dtypes(include=[dtype]).columns.tolist()
    if type_cols:
        print(f"\nColonnes de type {dtype} ({len(type_cols)}):")
        # Afficher par groupes de 5 pour meilleure lisibilité
        for i in range(0, len(type_cols), 5):
            group = type_cols[i:i+5]
            print(f"  {', '.join(group)}")

# Colonnes après encodage
print("\nB. COLONNES APRÈS ENCODAGE:")
encoded_cols = df_encoded.columns.tolist()
print(f"Nombre total: {len(encoded_cols)}")

# Afficher les colonnes par type de données
for dtype in ['object', 'int64', 'float64', 'category']:
    type_cols = df_encoded.select_dtypes(include=[dtype]).columns.tolist()
    if type_cols:
        print(f"\nColonnes de type {dtype} ({len(type_cols)}):")
        # Afficher par groupes de 5 pour meilleure lisibilité
        for i in range(0, len(type_cols), 5):
            group = type_cols[i:i+5]
            print(f"  {', '.join(group)}")

# 5. Nouvelles colonnes créées (détail des colonnes one-hot)
new_cols = [col for col in encoded_cols if col not in original_cols]
print("\nC. NOUVELLES COLONNES CRÉÉES PAR ONE-HOT ENCODING:")
print(f"Nombre total: {len(new_cols)}")

# Regrouper les colonnes par leur préfixe (variable d'origine)
prefix_groups = {}
for col in new_cols:
    # Identifier le préfixe (nom de variable d'origine)
    parts = col.split('_')
    if len(parts) > 1:
        prefix = parts[0]
        if prefix not in prefix_groups:
            prefix_groups[prefix] = []
        prefix_groups[prefix].append(col)

# Afficher les colonnes groupées par variable d'origine
for prefix, cols in prefix_groups.items():
    print(f"\n{prefix} ({len(cols)} colonnes):")
    for col in cols:
        # Extraire la valeur encodée (après le dernier underscore)
        value = col.split('_', 1)[1] if '_' in col else col
        print(f"  {col} (= {prefix} est {value})")

# 6. Colonnes supprimées (si drop_first=True)
if 'drop_first' in locals() and drop_first:
    print("\nD. COLONNES PREMIÈRES CATÉGORIES (implicitement supprimées):")
    for prefix, cols in prefix_groups.items():
        # Identifier les valeurs encodées
        encoded_values = [col.split('_', 1)[1] if '_' in col else col for col in cols]
        
        # Trouver les valeurs originales dans la colonne avant encodage
        if prefix in df_clean.columns:
            original_values = df_clean[prefix].unique()
            dropped_values = [val for val in original_values if str(val) not in encoded_values 
                             and str(val) + '_' not in ''.join(encoded_values)]
            
            if dropped_values:
                print(f"  {prefix}: {dropped_values[0] if dropped_values else 'Non identifié'}")

# 7. Afficher un tableau récapitulatif
print("\nE. TABLEAU RÉCAPITULATIF DES VARIABLES:")
summary_data = []

# Pour chaque colonne catégorielle originale
for col in categorical_columns:
    if col != 'customerID':  # Exclure customerID
        # Identifier les nouvelles colonnes créées
        col_dummies = [c for c in new_cols if c.startswith(col + '_')]
        # Identifier les valeurs uniques originales
        unique_vals = df_clean[col].unique()
        
        # Ajouter à notre tableau récapitulatif
        summary_data.append({
            'Variable originale': col,
            'Valeurs uniques': len(unique_vals),
            'Colonnes créées': len(col_dummies),
            'Exemples de colonnes': ', '.join(col_dummies[:3]) + (', ...' if len(col_dummies) > 3 else '')
        })

# Afficher le récapitulatif
if summary_data:
    from tabulate import tabulate
    try:
        print(tabulate(summary_data, headers="keys", tablefmt="grid"))
    except ImportError:
        # Si tabulate n'est pas disponible
        for data in summary_data:
            print(f"\n{data['Variable originale']}:")
            print(f"  Valeurs uniques: {data['Valeurs uniques']}")
            print(f"  Colonnes créées: {data['Colonnes créées']}")
            print(f"  Exemples: {data['Exemples de colonnes']}")

Encodage des variables binaires (Yes/No -> 1/0):


  Partner: encodage terminé
  Dependents: encodage terminé
  PhoneService: encodage terminé
  PaperlessBilling: encodage terminé
  Churn: encodage terminé

Variables catégorielles à encoder avec one-hot: ['customerID', 'gender', 'SeniorCitizen', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

COMPARAISON DÉTAILLÉE DES COLONNES AVANT/APRÈS ENCODAGE

A. COLONNES ORIGINALES:
Nombre total: 21

Colonnes de type object (13):
  customerID, gender, SeniorCitizen, MultipleLines, InternetService
  OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV
  StreamingMovies, Contract, PaymentMethod

Colonnes de type int64 (6):
  Partner, Dependents, tenure, PhoneService, PaperlessBilling
  Churn

Colonnes de type float64 (2):
  MonthlyCharges, TotalCharges

B. COLONNES APRÈS ENCODAGE:
Nombre total: 32

Colonnes de type object (1):
  customerID

Colonnes de type in

In [13]:
df.head(
)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [14]:
# Tableau d'encodage avec meilleure lisibilité
from IPython.display import display, HTML

# Créer un tableau HTML avec des styles améliorés pour la lisibilité
html_table = """
<h3>Tableau d'encodage détaillé pour le client 7590-VHVEG</h3>
<table border="1" style="border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; font-size: 14px;">
  <thead style="background-color: #4a4a4a; color: white; font-weight: bold; text-align: center;">
    <tr>
      <th style="padding: 8px;">Variable originale</th>
      <th style="padding: 8px;">Type original</th>
      <th style="padding: 8px;">Valeur originale</th>
      <th style="padding: 8px;">Traitement</th>
      <th style="padding: 8px;">Colonne encodée</th>
      <th style="padding: 8px;">Type encodé</th>
      <th style="padding: 8px;">Valeur encodée</th>
    </tr>
  </thead>
  <tbody>
"""

# Définir les données du tableau
data = [
    # ID du client
    ["customerID", "Identifiant", "7590-VHVEG", "Conservé", "customerID", "Identifiant", "7590-VHVEG"],
    
    # Variables démographiques
    ["gender", "Catégorielle", "Female", "One-hot", "gender_Male", "Binaire", "0"],
    ["SeniorCitizen", "Catégorielle", "0 (No)", "One-hot", "SeniorCitizen_Yes", "Binaire", "0"],
    ["Partner", "Binaire", "Yes", "Encodage binaire", "Partner", "Numérique binaire", "1"],
    ["Dependents", "Binaire", "No", "Encodage binaire", "Dependents", "Numérique binaire", "0"],
    
    # Variables d'abonnement
    ["tenure", "Numérique", "1", "Conservé", "tenure", "Numérique", "1"],
    ["PhoneService", "Binaire", "No", "Encodage binaire", "PhoneService", "Numérique binaire", "0"],
    ["MultipleLines", "Catégorielle", "No phone service", "One-hot", "MultipleLines_No phone service", "Binaire", "1"],
    ["MultipleLines", "Catégorielle", "No phone service", "One-hot", "MultipleLines_Yes", "Binaire", "0"],
    
    # Variables de service internet
    ["InternetService", "Catégorielle", "DSL", "One-hot", "InternetService_Fiber optic", "Binaire", "0"],
    ["InternetService", "Catégorielle", "DSL", "One-hot", "InternetService_No", "Binaire", "0"],
    ["OnlineSecurity", "Catégorielle", "No", "One-hot", "OnlineSecurity_No internet service", "Binaire", "0"],
    ["OnlineSecurity", "Catégorielle", "No", "One-hot", "OnlineSecurity_Yes", "Binaire", "0"],
    ["OnlineBackup", "Catégorielle", "Yes", "One-hot", "OnlineBackup_No internet service", "Binaire", "0"],
    ["OnlineBackup", "Catégorielle", "Yes", "One-hot", "OnlineBackup_Yes", "Binaire", "1"],
    ["DeviceProtection", "Catégorielle", "No", "One-hot", "DeviceProtection_No internet service", "Binaire", "0"],
    ["DeviceProtection", "Catégorielle", "No", "One-hot", "DeviceProtection_Yes", "Binaire", "0"],
    ["TechSupport", "Catégorielle", "No", "One-hot", "TechSupport_No internet service", "Binaire", "0"],
    ["TechSupport", "Catégorielle", "No", "One-hot", "TechSupport_Yes", "Binaire", "0"],
    ["StreamingTV", "Catégorielle", "No", "One-hot", "StreamingTV_No internet service", "Binaire", "0"],
    ["StreamingTV", "Catégorielle", "No", "One-hot", "StreamingTV_Yes", "Binaire", "0"],
    ["StreamingMovies", "Catégorielle", "No", "One-hot", "StreamingMovies_No internet service", "Binaire", "0"],
    ["StreamingMovies", "Catégorielle", "No", "One-hot", "StreamingMovies_Yes", "Binaire", "0"],
    
    # Variables de contrat
    ["Contract", "Catégorielle", "Month-to-month", "One-hot", "Contract_One year", "Binaire", "0"],
    ["Contract", "Catégorielle", "Month-to-month", "One-hot", "Contract_Two year", "Binaire", "0"],
    ["PaperlessBilling", "Binaire", "Yes", "Encodage binaire", "PaperlessBilling", "Numérique binaire", "1"],
    
    # Variables de paiement
    ["PaymentMethod", "Catégorielle", "Electronic check", "One-hot", "PaymentMethod_Credit card (automatic)", "Binaire", "0"],
    ["PaymentMethod", "Catégorielle", "Electronic check", "One-hot", "PaymentMethod_Electronic check", "Binaire", "1"],
    ["PaymentMethod", "Catégorielle", "Electronic check", "One-hot", "PaymentMethod_Mailed check", "Binaire", "0"],
    
    # Variables numériques
    ["MonthlyCharges", "Numérique", "29.85", "Conservé", "MonthlyCharges", "Numérique", "29.85"],
    ["TotalCharges", "Numérique", "29.85", "Conservé", "TotalCharges", "Numérique", "29.85"],
    
    # Variable cible
    ["Churn", "Binaire", "No", "Encodage binaire", "Churn", "Numérique binaire", "0"]
]

# Couleurs avec meilleur contraste pour la lisibilité
colors = {
    "Identifiant": "#FFF9C4;color:#333333",  # Jaune très pâle avec texte foncé
    "Catégorielle": "#FFCDD2;color:#333333", # Rouge pâle avec texte foncé
    "Binaire": "#C8E6C9;color:#333333",      # Vert pâle avec texte foncé
    "Numérique": "#BBDEFB;color:#333333",    # Bleu pâle avec texte foncé
    "Numérique binaire": "#C8E6C9;color:#333333" # Vert pâle avec texte foncé
}

# Ajouter des styles pour les groupes de lignes
row_styles = {
    0: "background-color: #f9f9f9;", # ID
    1: "background-color: #f0f0f0;", # Démographiques début
    5: "background-color: #f9f9f9;", # Services début
    25: "background-color: #f0f0f0;", # Contrat début
    30: "background-color: #f9f9f9;", # Variable cible
}

# Ajouter les lignes au tableau HTML avec alternance de couleurs
for idx, row in enumerate(data):
    # Déterminer si cette ligne commence un nouveau groupe
    row_style = ""
    if idx in row_styles:
        row_style = row_styles[idx]
    
    html_table += f"<tr style='{row_style}'>"
    
    for i, cell in enumerate(row):
        # Style pour les cellules de type
        cell_style = "padding: 8px; text-align: left;"
        if i == 1 or i == 5:  # colonnes de type
            for key, color_info in colors.items():
                if key in cell:
                    cell_style += f" background-color: {color_info.split(';')[0]}; {color_info.split(';')[1]};"
                    break
        
        # Style spécial pour la valeur encodée
        if i == 6:
            cell_style += " text-align: center; font-weight: bold;"
        
        html_table += f"<td style='{cell_style}'>{cell}</td>"
    
    html_table += "</tr>\n"

# Fermer la table et ajouter la légende
html_table += """
  </tbody>
</table>
<div style="margin-top: 20px;">
  <p style="font-weight: bold; margin-bottom: 10px;">Légende des types:</p>
  <div style="display: flex; flex-wrap: wrap; gap: 10px;">
    <div style="background-color: #FFF9C4; color: #333333; padding: 5px 10px; border-radius: 4px;">Identifiant</div>
    <div style="background-color: #FFCDD2; color: #333333; padding: 5px 10px; border-radius: 4px;">Catégorielle</div>
    <div style="background-color: #C8E6C9; color: #333333; padding: 5px 10px; border-radius: 4px;">Binaire</div>
    <div style="background-color: #BBDEFB; color: #333333; padding: 5px 10px; border-radius: 4px;">Numérique</div>
  </div>
</div>

<div style="margin-top: 20px;">
  <p style="font-weight: bold; margin-bottom: 10px;">Résumé:</p>
  <ul style="list-style-type: disc; padding-left: 20px;">
    <li>Variables originales: 21</li>
    <li>Variables après encodage: 32</li>
    <li>Variables identifiantes: 1</li>
    <li>Variables numériques: 3</li>
    <li>Variables binaires: 28</li>
  </ul>
</div>
"""

# Afficher le tableau HTML
display(HTML(html_table))

Variable originale,Type original,Valeur originale,Traitement,Colonne encodée,Type encodé,Valeur encodée
customerID,Identifiant,7590-VHVEG,Conservé,customerID,Identifiant,7590-VHVEG
gender,Catégorielle,Female,One-hot,gender_Male,Binaire,0
SeniorCitizen,Catégorielle,0 (No),One-hot,SeniorCitizen_Yes,Binaire,0
Partner,Binaire,Yes,Encodage binaire,Partner,Numérique binaire,1
Dependents,Binaire,No,Encodage binaire,Dependents,Numérique binaire,0
tenure,Numérique,1,Conservé,tenure,Numérique,1
PhoneService,Binaire,No,Encodage binaire,PhoneService,Numérique binaire,0
MultipleLines,Catégorielle,No phone service,One-hot,MultipleLines_No phone service,Binaire,1
MultipleLines,Catégorielle,No phone service,One-hot,MultipleLines_Yes,Binaire,0
InternetService,Catégorielle,DSL,One-hot,InternetService_Fiber optic,Binaire,0


In [ ]:
df_encoded.to_csv("CustomersFinal.csv", index=False)